# VI. Machine Learning Part I: The Tools

## Regression

Regression is a series of statistical tools to find the equation of a function which best fits some scattered data. We will generate data with an underlying trend, make a guess of this trend, and devise an iteration to improve this guess.

#### A Linear Trend

We will start with the simplest case in the book: a linear pattern. This is a chance for us to become acquainted with some of the fundamental working part of a neural network, which we will implement shortly.

First we import some tools:

In [ ]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt

We will begin by encoding the equation of a line
$$y=mx+b,$$
as a python function.

For validation, rather than using real data, we will generate our own. First we will fix the parameters of the linear equation; we will use capital letters, $M$ and $B$, to distinguish the *real* parameters from the *estimated* ones later.

Now, we generate the data. First, we make an array of $N$ values of the $x$-coordinate, $X_i$:

We will first define $L_i=mX_i+b$, the $y$-values which truly satisfy the linear relation. Then, we define noisy $y$-values by $Y_i=L_i+\sigma Z_i$, where $Z_i\sim U(-1,1)$, a uniform random variable.

#### Plotting The Data

We now have pairs $(X_i, Y_i)$ which represent data with a noisy linear trend. We can plot the pairs over the linear equation to visualise the trend with the following code:

In [ ]:
#Scatter plot
plt.scatter(X, Y, c='b', marker='.')

#Line plot
t = np.linspace(0,1,100)
plt.plot(t, linear(t, M, B), c='g')

#Labels
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Linear trend", "Noisy data"])

The first line simply makes a scatter plot of the $(X_i, Y_i)$ pairs, in blue. To plot the linear trend we create an array of ordered $x$-coordinates, `t`, using the `np.linspace` function; then we plot the `linear` function applied to `t` to present a "continuous" line.

#### Guessing the Trend

The data is noisy as intended, but the linear trend is clear. Suppose now we didn't know the values of $M$ and $B$. How could we approximate them? What we will do is, as usual, begin with a guess and come up with a way of improving it iteratively. First we guess approximate values of $M$ and $B$, and we call them $m$ and $b$:

We can now apply the linear equation with parameters $m$ and $b$ to $X_i$ to obtain our guesses for the $y$-coordinate, $y_i$. If our guess is good, the values of $y_i$ will be *close* to the real ordinates $Y_i$.

#### Comparing the Trends

We could show these values as a scatter plot, but the comparison is easier if we simply add the line with coefficients $m$ and $b$ to our previous plot.

In [ ]:
#Previous plots
plt.scatter(X, Y, marker='.')
plt.plot(t, linear(t, M, B), 'g')

#New plots
plt.plot(t, linear(t, m, b), 'r')

#Labels
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Linear trend", "Approximate trend", "Noisy data"])

Our initial guess for the trend is plotted in red, whereas the real trend is plotted in green. If you picked $m$ and $b$ somewhat randomly, the guess is probably not a very good one. We will now work to improve this.

#### The Loss Function

The first thing to do is to quantify the quality of the approximation. We need a numerical value, a **score**, which lets us compare different guesses; this is sometimes called a *cost function*, an *objective function*, or a **loss function**.

A typical loss function for trend estimation is the [*mean squared error* (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error):
$$
\mathcal{J} = \frac{1}{N}\sum_{i=1}^{N}(Y_i-y_i)^2 = \frac{1}{N}\sum_{i=1}^{N}(Y_i-mX_i-b)^2.
$$
This is a nice choice of loss function for a number of reasons. For instance, squaring the error makes small deviations even smaller whilst enlarging deviations greater than $1$. In a way, this loss function puts the emphasis on large erros and "forgives" the smaller deviations; it does not matter if the guess is inexact as long as no point is missed by a large margin. This is important because the noisy data is *not* exactly linear, even though it is *close* to linear.

We can define the `loss` function succinctly with a list comprehension and test the loss of the current guess:

#### Minima of the Loss Function

Now that we have a guess and its corresponding score, we should try to improve it. Another of the nice properties of the MSE loss function is that it is differentiable with respect to the parameters $m$ and $b$. We can compute the derivatives explicitly. By recalling the cost
\begin{align}
\mathcal{J} = \frac{1}{N}\sum_{i=1}^{N}(Y_i-y_i)^2 = \frac{1}{N}\sum_{i=1}^{N}(Y_i-mX_i-b)^2,
\end{align}
we obtain
\begin{align}
\frac{\partial\mathcal{J}}{\partial m}
& = \frac{-2}{N}\sum_{i=1}^{N}(Y_i-mX_i-b)X_i
= \frac{-2}{N}\sum_{i=1}^{N}(Y_i-y_i)X_i,
\\\frac{\partial\mathcal{J}}{\partial b}
& = \frac{-2}{N}\sum_{i=1}^{N}(Y_i-mX_i-b)
= \frac{-2}{N}\sum_{i=1}^{N}(Y_i-y_i).
\end{align}
Both of these derivatives should be zero once we find the minimisers of the loss function, i.e. the values of $m$ and $b$ which yield a minimum of the error. We will not consider the issue of several minima for now.

We can implement functions for the derivatives and check their current value:

#### Gradient Descent

The current derivatives are far from being zero, which means the loss is far from being minimal. We can use gradient descent to update our guesses.

In general, gradient descent is a method for finding a minimum of a function. If we are trying to minimise a function $f:\mathbb{R}^n\rightarrow \mathbb{R}$, we begin with an initial guess $x_0\in\mathbb{R}^n$ and compute the gradient of $f$ at that point, $\nabla f(x_0)$. We then update our guess by
$$
x_1 = x_0 - s_0\nabla f(x_0)
$$
for some positive constant $s_0$. In general, the iteration is
$$
x_{n+1} = x_n - s_n\nabla f(x_n).
$$
Note that if $f$ takes only a real argument, the gradient is simply the derivative:
$$
x_{n+1} = x_n - s_n\frac{\mathrm{d}f}{\mathrm{d}x}(x_n).
$$
The $s_n$ have to be chosen small enough to ensure that $f(x_{n+1})\leq f(x_{n})$. A typical approach is to fix $s$ and begin the descent checking that $f(x_{n+1})\leq f(x_{n})$ at every step. As soon as $f(x_{n+1})>f(x_{n})$, the current $s$ is rejected and a new, smaller constant is chosen.

In our case, thinking of $\mathcal{J}$ as a function of $m$ and $b$, the gradient is
$$
\nabla\mathcal{J}=\left(\frac{\partial\mathcal{J}}{\partial m},\frac{\partial\mathcal{J}}{\partial b}\right).
$$
The iteration will look like
$$
\left(m_{n+1},b_{n+1}\right)
= \left(m_{n},b_{n}\right)
- s_n\left(\frac{\partial\mathcal{J}}{\partial m},\frac{\partial\mathcal{J}}{\partial b}\right)
,
$$
which can be done separately for $m$ and for $b$ as
\begin{align}
m_{n+1} &= m_{n}- s_n\frac{\partial\mathcal{J}}{\partial m},
\\b_{n+1} &= b_{n}- s_n\frac{\partial\mathcal{J}}{\partial b}.
\end{align}

Like every iteration, we will need a stopping criterion. We will select a tolerance $\varepsilon$ and stop whenever $\left|\frac{\partial\mathcal{J}}{\partial m}\right|<\varepsilon$ and $\left|\frac{\partial\mathcal{J}}{\partial b}\right|<\varepsilon$.

We will now implement the descent algorithm, storing each value of the loss in a list `lossList` to plot it later.

We have convergence! The optimal values of $m$ and $b$ we recover might be a little different from $M$ and $B$, both because we only have an approximation, but also because the *true* linear trend of the data might be slightly different from the trend given by $M$ and $B$ since we only have a finite number of samples.

We can plot the estimated trend to see whether or not it captures the trend:

In [ ]:
plt.scatter(X, Y, marker='.')
plt.plot(t, linear(t, M, B), 'g')
plt.plot(t, linear(t, m, b), 'r')

plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Linear trend", "Approximate trend", "Noisy data"])

We can also plot the `lossList` over the number of iterations to see how it has decreased:

In [ ]:
plt.semilogy(range(0,len(lossList)),lossList)

plt.xlabel("Iterations")
plt.ylabel("Loss")

### Non-Linear Regression

The problem we have just considered, often called the [Linear Least Squares problem](https://en.wikipedia.org/wiki/Linear_least_squares), actually has an exact solution, easy to compute but not so easy to justify. However, the gradient descent method we have used is more general, and will let us deal with non-linear data as well.

This time we are going to assume the data has a quadratic trend, given by
$$
y = ax^2+bx+c.
$$
To generate some validation data, we will define the quadratic:

We choose the quadratic $10x^2-10x+5$, simply because it has a nice shape on the interval $[0,1]$. We define the true parameters, $A, B, C$:

Again, we generate random $x$-coordinates $X_i$,

and we also generate the true quadratic values $L_i=AX_i^2+BX_i+C$ as well as the noisy values $Y_i=L_i+\sigma Z_i$:

We proceed to plot the trend:

In [ ]:
#Scatter plot
plt.scatter(X, Y, c='b', marker='.')

#Quadratic plot
t = np.linspace(0,1,100)
plt.plot(t, quadratic(t, A, B, C), c='g')

#Labels
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Quadratic trend", "Noisy data"])

Now we are ready to make a guess, define the loss function, compute its derivatives and descend its gradient.
 
We can initialise the approximate parameters to anything in practice. As we saw before, the quadratic error causes the loss function to decrease rapidly at the beginning; the parameters will rapidly become *close* to the real ones, even if full convergence will take longer.

Once again we can evaluate the $y$-coordinates corresponding to our current guess and plot the guessed cubic trend to compare.

In [ ]:
plt.scatter(X, Y, marker='.')
plt.plot(t, quadratic(t, A, B, C), 'g')
plt.plot(t, quadratic(t, a, b, c), 'r')

#Labels
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Quadratic trend", "Approximate trend", "Noisy data"])

We will again define a MSE loss function,
$$
\mathcal{J} = \frac{1}{N}\sum_{i=1}^{N}(Y_i-y_i)^2 = \frac{1}{N}\sum_{i=1}^{N}\left[Y_i-(aX_i^2+bX_i+c)\right]^2,
$$
and compute its derivatives with respect to $a, b$ and $c$:
\begin{align}
\frac{\partial\mathcal{J}}{\partial a}
& = \frac{-2}{N}\sum_{i=1}^{N}\left[Y_i-(aX_i^2+bX_i+c)\right]X_i^2
= \frac{-2}{N}\sum_{i=1}^{N}(Y_i-y_i)X_i^2
,\\\frac{\partial\mathcal{J}}{\partial b}
& = \frac{-2}{N}\sum_{i=1}^{N}\left[Y_i-(aX_i^2+bX_i+c)\right]X_i
= \frac{-2}{N}\sum_{i=1}^{N}(Y_i-y_i)X_i
,\\\frac{\partial\mathcal{J}}{\partial c}
& = \frac{-2}{N}\sum_{i=1}^{N}\left[Y_i-(aX_i^2+bX_i+c)\right]
= \frac{-2}{N}\sum_{i=1}^{N}(Y_i-y_i)
.
\end{align}

We can implement these &mdash; in fact we can recycle some of the code from the linear regression section!

The gradient descent algorithm for the problem looks very similar to the previous one:
\begin{align}
a_{n+1} &= a_{n}- s_n\frac{\partial\mathcal{J}}{\partial a},
\\b_{n+1} &= b_{n}- s_n\frac{\partial\mathcal{J}}{\partial b},
\\c_{n+1} &= c_{n}- s_n\frac{\partial\mathcal{J}}{\partial c},
\\d_{n+1} &= d_{n}- s_n\frac{\partial\mathcal{J}}{\partial d}.
\end{align}

All that remains now is the gradient descent loop, which will also look very similar to the previous loop:

The convergence is significantly slower than it was in the linear case. In the end, however, we obtain a nice fit:

In [ ]:
plt.scatter(X, Y, marker='.')
plt.plot(t, quadratic(t, A, B, C), 'g')
plt.plot(t, quadratic(t, a, b, c), 'r')

plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Quadratic trend", "Approximate trend", "Noisy data"])

plt.figure()

plt.semilogy(range(0,len(lossList)),lossList)
plt.xlabel("Iterations")
plt.ylabel("Loss")

## Sometimes, Gradients Won't Do

### The Cubic Polynomial

The blocks below implement the gradient descent for the cubic fitting problem, 
$$
y = ax^3+bx^2+cx+d,
$$
and proceed just as before. Run them and look at the results of the descent. Here are the functions:

In [ ]:
# Functions

def cubic(x, a, b, c, d): return a*x**3 + b*x**2 + c*x + d
def loss(N, y, X, Y): return sum([(Y[k]-y[k])**2 for k in range(0, N)])/N
def aDerivative(N, y, X, Y): return -2*sum([(Y[k]-y[k])*X[k]**3 for k in range(0, N)])/N
def bDerivative(N, y, X, Y): return -2*sum([(Y[k]-y[k])*X[k]**2 for k in range(0, N)])/N
def cDerivative(N, y, X, Y): return -2*sum([(Y[k]-y[k])*X[k] for k in range(0, N)])/N
def dDerivative(N, y, X, Y): return -2*sum([(Y[k]-y[k]) for k in range(0, N)])/N

Here are the parameters:

In [ ]:
# Parameters

A = 10
B = -15
C = 5
D = 2

N = 200
X = rand(N)

L = cubic(X, A, B, C, D)

sigma = 0.3
Y = L + (rand(N)*2-1)*sigma

a = (rand()*2-1)*10
b = (rand()*2-1)*10
c = (rand()*2-1)*10
d = (rand()*2-1)*10

Note that we have chosen initial guesses which are very close to the real values! Here is the plotting:

In [ ]:
#Plotting

plt.scatter(X, Y, marker='.')
plt.plot(t, cubic(t, A, B, C, D), 'g')
plt.plot(t, cubic(t, a, b, c, d), 'r')
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Cubic trend", "Approximate trend", "Noisy data"])

Finally, here is the main loop:

In [ ]:
a = (rand()*2-1)*10
b = (rand()*2-1)*10
c = (rand()*2-1)*10
d = (rand()*2-1)*10

y = cubic(X, a, b, c, d)

s = 1000
tol = 1.0e-8

lossCurrent = loss(N, y, X, Y)
lossList = [lossCurrent]

for k in range(0, 10000):
    aDer = aDerivative(N, y, X, Y)
    bDer = bDerivative(N, y, X, Y)
    cDer = cDerivative(N, y, X, Y)
    dDer = dDerivative(N, y, X, Y)
    
    aDerAbs = abs(aDer)
    bDerAbs = abs(bDer)
    cDerAbs = abs(cDer)
    dDerAbs = abs(dDer)
    
    aNew = a - s*aDer
    bNew = b - s*bDer
    cNew = c - s*cDer
    dNew = d - s*dDer
    
    yNew = cubic(X, aNew, bNew, cNew, dNew)

    lossNew = loss(N, yNew, X, Y)

    if lossNew > lossCurrent:
        s = s/2
        print(f"k = {k}, descent failed. New step = {s}")
    else:
        a = aNew
        b = bNew
        c = cNew
        d = dNew
        y = yNew
        
        lossCurrent = lossNew
        lossList.append(lossCurrent)
        print(f"k = {k}. Loss = {lossNew}.")
    
    if aDerAbs<tol and bDerAbs<tol and cDerAbs<tol and dDerAbs<tol:
        break

print(f"\n True values: A = {A}, B = {B}, C = {C}, D = {D}.")
print(f"\n Approximate values: a = {a}, b = {b}, c = {c}, d = {d}.")

plt.scatter(X, Y, marker='.')
plt.plot(t, cubic(t, A, B, C, D), 'g')
plt.plot(t, cubic(t, a, b, c, d), 'r')
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Cubic trend", "Approximate trend", "Noisy data"])

plt.figure()
plt.semilogy(range(0,len(lossList)),lossList)
plt.xlabel("Iterations")
plt.ylabel("Loss")

What went wrong? Well... nothing. The algorithm has run for 10000 iterations and simply has not converged yet. The choice of parameters have improved, the guess is *close* to the original trend, the loss has decreased over time, but the algorithm simply isn't done yet because the optimisation in four parameters is slow. **Gradient descent is slow**.

### The Barzilai-Borwein Update

One of the reasons the previous algorithm was very slow is that the step $s_n$ was not set in a very thoughtful way; once it decreased, it was not allowed to increase again. We will use this type of gradient descent to implement a neural network in the future, but you should know that there are more sophisticated optimisation methods out there. You cannot learn them all in one lesson, but here we will explore some code examples.

The [Barzilai-Borwein update](https://doi.org/10.1093/imanum/8.1.141) is a **very** clever choice of the steps $s_n$ which greatly accelerates the gradient descent. If we are trying to minimise a function $f:\mathbb{R}^n\rightarrow \mathbb{R}$ with the gradient descent iteration
$$
x_{n+1} = x_n - s_n\nabla f(x_n),
$$
we define the Barzilai-Borwein update by either
$$
s_n=\frac{
\langle
x_{n}-x_{n-1}
,
\nabla f(x_{n})-\nabla f(x_{n-1})
\rangle
}{
\|\nabla f(x_{n})-\nabla f(x_{n-1})\|^2
}
$$
or
$$
s_n=\frac{
\|x_{n}-x_{n-1}\|^2
}{
\langle
x_{n}-x_{n-1}
,
\nabla f(x_{n})-\nabla f(x_{n-1})
\rangle
}.
$$

To find out why this is a good choice of the step you can go to the [original paper](https://doi.org/10.1093/imanum/8.1.141). However, we can verify this numerically. The code below implements the gradient descent optimisation of the same cubic problem we attempted before, only this time the step $s_n$ is chosen according to this new rule. Try to run it:

In [ ]:
a = (rand()*2-1)*10
b = (rand()*2-1)*10
c = (rand()*2-1)*10
d = (rand()*2-1)*10

y = cubic(X, a, b, c, d)

s = 0.01
tol = 1.0e-8

lossCurrent = loss(N, y, X, Y)
lossList = [lossCurrent]

for k in range(0, 1000):
    aDer = aDerivative(N, y, X, Y)
    bDer = bDerivative(N, y, X, Y)
    cDer = cDerivative(N, y, X, Y)
    dDer = dDerivative(N, y, X, Y)
    
    aDerAbs = abs(aDer)
    bDerAbs = abs(bDer)
    cDerAbs = abs(cDer)
    dDerAbs = abs(dDer)
    
    # Barzilai-Borwein
    if k>0:
        num = (a-aOld)**2+(b-bOld)**2+(c-cOld)**2+(d-dOld)**2
        den = (a-aOld)*(aDer-aDerOld)+(b-bOld)*(bDer-bDerOld)+(c-cOld)*(cDer-cDerOld)+(d-dOld)*(dDer-dDerOld)
        s = num/den
    
    aNew = a - s*aDer
    bNew = b - s*bDer
    cNew = c - s*cDer
    dNew = d - s*dDer
    
    yNew = cubic(X, aNew, bNew, cNew, dNew)

    lossNew = loss(N, yNew, X, Y)

    aOld = a
    bOld = b
    cOld = c
    dOld = d
    
    a = aNew
    b = bNew
    c = cNew
    d = dNew
    y = yNew
        
    aDerOld = aDer
    bDerOld = bDer
    cDerOld = cDer
    dDerOld = dDer
    
    lossCurrent = lossNew
    lossList.append(lossCurrent)
    print(f"k = {k}. Loss = {lossNew}.")
    
    if aDerAbs<tol and bDerAbs<tol and cDerAbs<tol and dDerAbs<tol:
        break

print(f"\n True values: A = {A}, B = {B}, C = {C}, D = {D}.")
print(f"\n Approximate values: a = {a}, b = {b}, c = {c}, d = {d}.")

plt.scatter(X, Y, marker='.')
plt.plot(t, cubic(t, A, B, C, D), 'g')
plt.plot(t, cubic(t, a, b, c, d), 'r')
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Cubic trend", "Approximate trend", "Noisy data"])

plt.figure()
plt.semilogy(range(0,len(lossList)),lossList)
plt.xlabel("Iterations")
plt.ylabel("Loss")

Beautiful convergence in just a few iterations!

Notice that the loss function does not decrease monotonically. This method is clearly doing something more complicated than simple descent. Remember this as a nice trick for the future.